In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Utah FORGE Well 58-32 Processed Log
# Replace with your local path if you have downloaded the CSV
df = pd.read_csv("Well_58-32_processed_pason_log.csv")

# Initial Inspection
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7311 entries, 0 to 7310
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Depth(ft)              7311 non-null   float64
 1   Depth(m)               7311 non-null   float64
 2   ROP(1 ft)              7311 non-null   float64
 3   ROP(1 m)               7311 non-null   float64
 4   weight on bit (k-lbs)  7311 non-null   float64
 5   weight on bit (kg)     7311 non-null   float64
 6   Temp Out( degF)        7311 non-null   float64
 7   Temp Out( degC)        7311 non-null   float64
 8   Temp In(degF)          7311 non-null   float64
 9   Temp In(degC)          7311 non-null   float64
 10  Pit Total (bbls)       7311 non-null   float64
 11  Pit Total (m3)         7311 non-null   float64
 12  Pump Press (psi)       7311 non-null   float64
 13  Pump Press (KPa)       7311 non-null   float64
 14  Hookload (k-lbs)       7311 non-null   float64
 15  Hook

,Depth(ft),Depth(m),ROP(1 ft),ROP(1 m),weight on bit (k-lbs),weight on bit (kg),Temp Out( degF),Temp Out( degC),Temp In(degF),Temp In(degC),...,Surface Torque (KPa),Rotary Speed (rpm),Flow In (gal/min),Flow In(liters/min),Flow Out %,WH Pressure (psi),WH Pressure (KPa),H2S Floor,H2S Cellar,H2S Pits
0,85.18,25.96,56.80,17.31,0.0,0.0,93.49,34.16,90.87,32.71,...,61.57,151.09,1109.58,4200.22,101.61,-1230.69,-8485.61,-0.01,-0.03,0.31
1,86.18,26.27,61.14,18.63,0.0,0.0,93.52,34.18,90.83,32.68,...,61.57,151.09,1108.36,4195.60,101.70,-1230.69,-8485.61,-0.01,-0.04,0.31
2,87.23,26.59,18.58,5.66,0.0,0.0,94.25,34.58,90.91,32.73,...,62.12,61.54,1102.87,4174.82,96.81,-1230.69,-8485.61,-0.01,-0.03,0.30
3,88.26,26.90,38.29,11.67,0.0,0.0,94.44,34.69,91.14,32.86,...,61.85,60.23,1101.04,4167.89,96.67,-1230.69,-8485.61,0.00,-0.03,0.31
4,89.36,27.24,394.38,120.20,0.0,0.0,94.48,34.71,91.18,32.88,...,61.85,61.75,1102.46,4173.26,96.63,-1230.69,-8485.61,-0.02,-0.02,0.31
